In [1]:
import pandas as pd
import csv
from scipy.sparse import csr_matrix
from datetime import datetime
import numpy as np

In [2]:
train = csv.reader((open('train.csv')))
next(train, None)

# dictionary with (user, [(artist1, numplays1), (artist2, numplays2) ...])
user_artist = {}

# row[0] = user id, row[1] = artist id
# this is fast; takes < 1 minute
print 'Begin:', str(datetime.now())
for i, row in enumerate(train):
	if user_artist.get(row[0]) == None:
		user_artist[row[0]] = [(row[1], row[2])]
	else:
		user_artist[row[0]].append((row[1], row[2]))
	if i % 1000000 == 0:
		print 'Row', i

print 'End:', str(datetime.now())

Begin: 2017-04-05 20:22:19.637137
Row 0
Row 1000000
Row 2000000
Row 3000000
Row 4000000
End: 2017-04-05 20:22:34.500483


In [2]:
users = pd.read_csv('profiles.csv')

In [3]:
country_1hot = pd.get_dummies(users['country'])
users = users.drop('country', axis=1)

In [4]:
users = pd.concat([users, country_1hot], axis=1)

In [5]:
sex_1hot = pd.get_dummies(users['sex'])
users = users.drop('sex', axis=1)

In [6]:
users = pd.concat([users, sex_1hot], axis=1)

In [7]:
def nan(x):
    if x < 0 or x > 90:
        return np.nan
    return x

users['age'] = users['age'].apply(nan)

In [9]:
median_age = users['age'].median(axis=0)

In [10]:
def median_fxn(x):
    if np.isnan(x):
        return median_age
    return x
    
users['age'] = users['age'].apply(median_fxn)

In [11]:
users[np.isnan(users['Afghanistan'])]

,user,age,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,...,Viet Nam,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,f,m


In [12]:
for col in users.columns.values[range(1,len(users.columns))]:
    users[col] = users[col].astype(int)

In [14]:
users['age'] = users['age'].astype(int)

In [17]:
users.to_csv('one-hot-profiles.csv', index=False)